## Introduction to Data Science

### Dashboarding and interactive data

#### Based on [this Plotly post](https://medium.com/@drimik99/a-python-programmers-guide-to-dashboarding-part-1-8db0c48eee9d), [this streamlit post](https://towardsdatascience.com/coding-ml-tools-like-you-code-ml-models-ddba3357eace) and these comparisons [ (1) ](https://medium.datadriveninvestor.com/streamlit-vs-dash-vs-voil%C3%A0-vs-panel-battle-of-the-python-dashboarding-giants-177c40b9ea57#8026) and [ (2) ](https://towardsdatascience.com/plotly-dash-vs-streamlit-which-is-the-best-library-for-building-data-dashboard-web-apps-97d7c98b938c)between platforms

+ #### [Dash Plotly](https://plotly.com/dash/)  
+ #### [Streamlit](https://streamlit.io/)  
+ #### [Voila](https://github.com/voila-dashboards/voila)  
+ #### [Panel](https://pythonrepo.com/repo/holoviz-panel-python-administrative-interfaces)  

### Dash Plotly

#### Structure of the code

Imports

In [2]:
import json
import datetime
import operator
import os
import base64
import io

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

import dash
import dash.dependencies
import dash_html_components as html
import dash_core_components as dcc
import dash_table_experiments as dte
from dash.dependencies import Input, Output, State

import plotly

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#### Application

In [3]:
app = dash.Dash()
app.scripts.config.serve_locally = True
app.config['suppress_callback_exceptions'] = True

#### Layout

In [4]:
app.layout = html.Div([html.H5("Upload Files"),
                       dcc.Upload(id='upload-data',
                                  children=html.Div(['Drag and Drop or ',
                                                     html.A('Select Files')
                                                    ]),
                                  style={'width': '100%',
                                         'height': '60px',
                                         'lineHeight': '60px',
                                         'borderWidth': '1px',
                                         'borderStyle': 'dashed',
                                         'borderRadius': '5px',
                                         'textAlign': 'center',
                                         'margin': '10px'},
                                  multiple=False),
                       html.Br(),
                       html.Button(id='propagate-button',
                                   n_clicks=0,
                                   children='Propagate Table Data'),
                       html.Br(),
                       html.H5("Filter Column"),
                       dcc.Dropdown(id='dropdown_table_filterColumn',
                                    multi = False,
                                    placeholder='Filter Column'),
                       html.Br(),
                       html.H5("Updated Table"),
                       html.Div(dte.DataTable(rows=[{}], id='table'))
])

#### Functions

In [5]:
# file upload function
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))

    except Exception as e:
        print(e)
        return None
    return df

In [7]:
# callback table creation
@app.callback(Output('table', 'rows'),
              [Input('upload-data', 'contents'),
               Input('upload-data', 'filename')])
def update_output(contents, filename):
    if contents is not None:
        df = parse_contents(contents, filename)
        if df is not None:
            return df.to_dict('records')
        else:
            return [{}]
    else:
        return [{}]


#callback update options of filter dropdown
@app.callback(Output('dropdown_table_filterColumn', 'options'),
              [Input('propagate-button', 'n_clicks'),
               Input('table', 'rows')])
def update_filter_column_options(n_clicks_update, tablerows):
    if n_clicks_update < 1:
        print("df empty")
        return []

    else:
        dff = pd.DataFrame(tablerows) # <- problem! dff stays empty even though table was uploaded
        print("updating... dff empty?:", dff.empty) #result is True, labels stay empty
        return [{'label': i, 'value': i} for i in sorted(list(dff))]

#### CSS and start application

In [ ]:
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

if __name__ == '__main__':
    app.run_server(debug=True)